# 1. Import packages

In [1]:
import re
import string
from typing import Final, List

from IPython.core.display import display_html, HTML

from textattack import Attack, AttackArgs, Attacker
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions.classification import UntargetedClassification
from textattack.loggers import CSVLogger
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attacked_text import AttackedText
from textattack.transformations import WordSwap

/home/sypark/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load model and dataset

In [2]:
# Load from my_model.py and my_dataset.py
from my_model import get_model
from my_dataset import dataset

model = get_model("cuda:0")

# 3. Define `CustomWordSwapEmbedding` class

In [3]:
class CustomWordSwapEmbedding(WordSwap):
    def __init__(self) -> None:
        hexword_regex: Final[re.Pattern] = re.compile(r"^[0-9a-fA-F]{4}$")
        self.replacement_words: List[str] = []

        with open("./models/encryptd_vocab.txt") as vocab_file:
            for vocab in vocab_file:
                self.replacement_words.append(
                    vocab.rstrip()
                ) if hexword_regex.fullmatch(vocab.rstrip()) else None
                continue
            pass

        return super().__init__(string.hexdigits)

    def _get_transformations(
        self, current_text: AttackedText, indices_to_modify: List[int]
    ) -> List[AttackedText]:
        words = current_text.words
        transformed_texts: List[AttackedText] = []

        for i in indices_to_modify:
            # if i < 22:
            #     continue
            
            word_to_replace = words[i]
            replacement_words = self._get_replacement_words(word_to_replace)
            transformed_texts_idx: List[AttackedText] = []

            for r in replacement_words:
                if r == word_to_replace:
                    continue

                indices: List[int] = []
                new_words: List[str] = []

                indices.append(i)
                new_words.append(r)

                if i < len(words) - 1:
                    post_r = r[2:] + words[i + 1][2:]
                    indices.append(i + 1)
                    new_words.append(post_r)
                    pass

                transformed_texts_idx.append(
                    current_text.replace_words_at_indices(indices, new_words)
                )
                continue

            transformed_texts.extend(transformed_texts_idx)
            continue

        return transformed_texts

    def _get_replacement_words(self, word: str) -> List[str]:
        replacement_words = [w for w in self.replacement_words if w.startswith(word[:2])]

        if word in replacement_words:
            replacement_words.remove(word)
            pass
        
        return replacement_words

    pass

# 4. Construct `textattack.Attack` object

In [4]:
# Construct the four fundamental components of the attack
goal_function = UntargetedClassification(model)
constraints = [
    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.9),
]
transformation = CustomWordSwapEmbedding()
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'my_model.Classifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# 5. Construct `textattack.Attacker` object

In [5]:
# Attack until 1000 successful attacks are reached
attack_args = AttackArgs(num_successful_examples=100, random_seed=42)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CustomWordSwapEmbedding
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  1%|          | 1/100 [00:02<03:25,  2.07s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<03:58,  2.41s/it]

[[11 (100%)]] --> [[2 (96%)]]

1191 9155 5530 303e 3e31 316a 6a78 78d7 d798 9850 5010 [[1000]] [[00ed]] edf4 f471 7100 0000 00cc cc24 245f 5f16 166b 6b52 525e 5e2c 2cd2 d2df dfdc dc7d 7de7 e712 12bf bf02 020d 0d18 1834 3442 4202 02a9 a93d 3dd0 d070 70ff ff0e 0e18 1819 1934 34e0 e0b0 b0f4 f4cb cb1f 1fa5 a5de de3e 3e13 1366 6637 378b 8bb9 b928 28dd dd9e 9e81

1191 9155 5530 303e 3e31 316a 6a78 78d7 d798 9850 5010 [[1014]] [[14ed]] edf4 f471 7100 0000 00cc cc24 245f 5f16 166b 6b52 525e 5e2c 2cd2 d2df dfdc dc7d 7de7 e712 12bf bf02 020d 0d18 1834 3442 4202 02a9 a93d 3dd0 d070 70ff ff0e 0e18 1819 1934 34e0 e0b0 b0f4 f4cb cb1f 1fa5 a5de de3e 3e13 1366 6637 378b 8bb9 b928 28dd dd9e 9e81




...

--------------------------------------------- Result 98 ---------------------------------------------
[[7 (100%)]] --> [[6 (99%)]]

[[0016]] [[168f]] 8f6a 6a0c 0cbd bd84 8491 911b 1b21 2180 8010 1001 0117 1761 6173 7300 0000 0001 0101 0108 080a [[0a00]] [[0039]] 3946 467c 7c27 2731 316f 6f28 28b7 b749 4910 1020 20cd cdad ad55 5590 9096 968b 8b2f 2f5f 5f23 23f1 f1fd fd16 1650 503c 3c9b 9b16 16fb fb34 34a6 a692 920b 0b4c 4caa aacc ccde deef ef82 82a4 a478 78e2 e2f8

[[00b2]] [[b28f]] 8f6a 6a0c 0cbd bd84 8491 911b 1b21 2180 8010 1001 0117 1761 6173 7300 0000 0001 0101 0108 080a [[0a74]] [[7439]] 3946 467c 7c27 2731 316f 6f28 28b7 b749 4910 1020 20cd cdad ad55 5590 9096 968b 8b2f 2f5f 5f23 23f1 f1fd fd16 1650 503c 3c9b 9b16 16fb fb34 34a6 a692 920b 0b4c 4caa aacc ccde deef ef82 82a4 a478 78e2 e2f8




[Succeeded / Failed / Skipped / Total] 99 / 0 / 0 / 99:  99%|█████████▉| 99/100 [04:26<00:02,  2.69s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[3 (100%)]] --> [[1 (93%)]]

[[c065]] [[6500]] 002c 2cc8 c845 4500 00d0 d01d 1d61 6121 211e 1eca caa5 a519 1988 882c 2cb1 b171 7191 9151 5179 7943 43b4 b47f 7f30 3025 2599 9944 4470 70a1 a1c6 c653 5333 33cd cdf4 f49d 9d9b 9b45 4587 87ed edb6

[[c045]] [[4500]] 002c 2cc8 c845 4500 00d0 d01d 1d61 6121 211e 1eca caa5 a519 1988 882c 2cb1 b171 7191 9151 5179 7943 43b4 b47f 7f30 3025 2599 9944 4470 70a1 a1c6 c653 5333 33cd cdf4 f49d 9d9b 9b45 4587 87ed edb6




[Succeeded / Failed / Skipped / Total] 100 / 0 / 0 / 100: 100%|██████████| 100/100 [04:29<00:00,  2.70s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[14 (100%)]] --> [[15 (100%)]]

17af af00 [[0000]] [[0001]] 0101 0108 080a [[0a00]] [[0022]] 2217 1739 391a 1aa4 a4cd cd77 7784 8415 1578 78a8 a8d2 d255 551a 1a96 9637 37fc fc7c 7cf6 f651 5100 001b 1bd7 d7ce ce90 90f4 f423 237c 7c66 66cb cb31 311c 1c46 46a1 a1d7 d76b 6b8f 8f4f 4fbd bdc5 c583 8343 431b 1b31 313c 3c2c 2ca6 a6ff ff31 3128 2894 948d 8d49 496f 6fea ea26

17af af00 [[001d]] [[1d01]] 0101 0108 080a [[0a9d]] [[9d22]] 2217 1739 391a 1aa4 a4cd cd77 7784 8415 1578 78a8 a8d2 d255 551a 1a96 9637 37fc fc7c 7cf6 f651 5100 001b 1bd7 d7ce ce90 90f4 f423 237c 7c66 66cb cb31 311c 1c46 46a1 a1d7 d76b 6b8f 8f4f 4fbd bdc5 c583 8343 431b 1b31 313c 3c2c 2ca6 a6ff ff31 3128 2894 948d 8d49 496f 6fea ea26



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 100    |
| Number of fail

# 6. Display attack results

In [6]:
# Display the attack results and the differences
logger = CSVLogger(color_method="html")
for result in attack_results:
    logger.log_attack_result(result)
    continue

display_html(HTML(logger.df[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,1191 9155 5530 303e 3e31 316a 6a78 78d7 d798 9850 5010 1000 00ed edf4 f471 7100 0000 00cc cc24 245f 5f16 166b 6b52 525e 5e2c 2cd2 d2df dfdc dc7d 7de7 e712 12bf bf02 020d 0d18 1834 3442 4202 02a9 a93d 3dd0 d070 70ff ff0e 0e18 1819 1934 34e0 e0b0 b0f4 f4cb cb1f 1fa5 a5de de3e 3e13 1366 6637 378b 8bb9 b928 28dd dd9e 9e81,1191 9155 5530 303e 3e31 316a 6a78 78d7 d798 9850 5010 1014 14ed edf4 f471 7100 0000 00cc cc24 245f 5f16 166b 6b52 525e 5e2c 2cd2 d2df dfdc dc7d 7de7 e712 12bf bf02 020d 0d18 1834 3442 4202 02a9 a93d 3dd0 d070 70ff ff0e 0e18 1819 1934 34e0 e0b0 b0f4 f4cb cb1f 1fa5 a5de de3e 3e13 1366 6637 378b 8bb9 b928 28dd dd9e 9e81
1,3514 1400 0000 0001 0101 0108 080a 0a1a 1aa6 a62a 2ac6 c600 0023 2374 7445 45f5 f5f3 f3a8 a81c 1c67 6752 523a 3afd fd91 91eb eb72 726e 6e13 13ee eef4 f45f 5f6d 6de3 e3d1 d192 9298 9828 281d 1db1 b1a5 a53d 3d90 9055 5501 017c 7c52 52fc fc9b 9b60 60b5 b5d2 d2db db57 57fc fc45 45e2 e247 477e 7edb db02 0219 19a7 a7b7 b7fb,3514 1400 0000 0001 0101 0108 080a 0a9a 9aa6 a62a 2ac6 c600 001e 1e74 7445 45f5 f5f3 f3ab ab1c 1c67 6752 523a 3afd fd91 91d7 d7e9 e96e 6e13 13ee eef4 f45f 5f6d 6d28 28d1 d1de de98 9828 281d 1db1 b1a5 a53d 3d90 9055 5501 017c 7c52 52fc fc9b 9b60 60b5 b5d2 d2db db57 57fc fc45 45e2 e247 477e 7edb db02 0219 19a7 a7b7 b7fb
2,03e1 e19d 9db2 b255 5523 2351 51ad ad40 4072 7250 5018 1896 96c8 c808 0829 2900 0000 0017 1703 0303 0300 0028 2800 0000 0000 0000 0000 0000 0000 0035 35f4 f42d 2d9c 9ccb cb7b 7b9a 9ae8 e804 040f 0f16 163c 3cd2 d2e5 e501 011e 1e4e 4ea1 a183 83e2 e221 21b1 b167 6726 26b5 b500 007c 7ce4 e4ff ff77 7787 87f4 f4ef,03ae ae9d 9db2 b255 5523 2351 51ad ad40 4072 7250 5018 1896 96c8 c808 0829 2900 0000 0017 1703 0303 0300 0028 2800 0000 0000 0000 0000 0000 0000 0035 35f4 f42d 2d9c 9ccb cb7b 7b9a 9ae8 e804 040f 0f16 163c 3cd2 d2e5 e501 011e 1e4e 4ea1 a183 83e2 e221 21b1 b167 6726 26b5 b500 007c 7ce4 e4ff ff77 7787 87f4 f4ef
3,1166 6628 283c 3ce0 e04c 4cb4 b468 6855 553f 3f50 5010 1000 00ed ed52 524a 4a00 0000 0040 4038 383d 3d3c 3c17 1780 8066 6604 04c7 c709 095b 5b14 14e3 e3f7 f727 2761 61de def2 f223 2386 86ac acd9 d909 09e1 e1de de85 85a7 a7a9 a916 1667 67d3 d337 37d9 d994 94cd cdd8 d87a 7af2 f2cd cdc2 c20a 0ab3 b321 21b6 b616 163d 3dd1,1166 6628 283c 3ce0 e04c 4cb4 b468 6855 553f 3f50 5010 1037 37ed ed52 524a 4a00 0000 0040 4038 383d 3d3c 3c17 1780 8066 6604 04c7 c709 095b 5b14 14e3 e3f7 f727 2761 61de def2 f223 2386 86ac acd9 d909 09e1 e1de de85 85a7 a7a9 a916 1667 67d3 d337 37d9 d994 94cd cdd8 d87a 7af2 f2cd cdc2 c20a 0ab3 b321 21b6 b616 163d 3dd1
4,e00e 0e00 0000 0001 0101 0108 080a 0a1a 1aa5 a5db db7a 7a00 0023 2325 2501 0143 43c9 c916 1699 9911 1193 939f 9f44 4420 20f9 f93f 3fdb db18 18c9 c98c 8c4a 4a30 30aa aa6d 6d2b 2ba5 a5d0 d081 818f 8f45 457c 7ceb ebad ad11 1124 2443 4363 63bd bdc0 c0a2 a267 679f 9f59 59c8 c892 9234 346a 6a93 93a1 a110 108a 8a82 82f6 f682,e00e 0e00 0000 0001 0101 0130 300a 0a9a 9aa5 a5db db7a 7a00 0023 2325 2501 0143 43c9 c916 1699 9911 1193 939f 9f44 4420 2055 553f 3fdb db18 18c9 c98c 8cd6 d630 30aa aa6d 6d2b 2ba5 a5d0 d081 818f 8f45 457c 7ceb ebad ad11 1124 2443 4363 63b5 b5c0 c0a2 a267 679f 9f59 59c8 c892 9234 346a 6a93 93a1 a110 108a 8a82 82f6 f682
5,8570 708f 8f9d 9d34 3488 8835 3579 798b 8b0d 0d80 8010 100c 0c69 69b7 b712 1200 0000 0001 0101 0108 080a 0a42 42a3 a3be be62 6200 00f2 f233 3328 284a 4ab4 b4c8 c8a1 a1a7 a77c 7cb1 b1ac acbd bd69 6931 31c4 c485 852b 2b72 720c 0cab ab60 60a3 a37b 7b90 90b3 b310 104b 4ba5 a5c4 c43e 3e0e 0eb2 b2cb cb4f 4fec ec1d 1d0d 0d6f,8515 158f 8f9d 9d34 3488 8835 3596 968b 8b0d 0d80 8010 100c 0c69 69b7 b712 1200 0000 0001 0101 0108 080a 0a77 77a3 a3be be62 6200 00f2 f233 3328 284a 4a79 79c8 c8a1 a1a7 a77c 7cb1 b1ac acbd bd69 6931 31c4 c485 852b 2b72 720c 0cab ab60 60a3 a37b 7b90 90b3 b310 100d 0da5 a5c4 c43e 3e0e 0eb2 b2cb cb4f 4fec ec1d 1d0d 0d6f
6,7f40 40af af6a 6a9e 9ef6 f64a 4a09 098b 8be0 e050 5018 1807 07fb fb8d 8d08 0800 0000 0017 1703 0303 0300 0024 245c 5cad ad3d 3d08 08fa f